In [1]:
oldpath = '../Human_anno/final'
newpath = '../Human_anno_sora_test'
sora_overall = '../GPT4o_eval_results\overall_consistency/overall_consistency_multiagent_ev_results-sora-1.json'

import os
import json

# with open(sora_overall, 'r') as f:
#     sora_eavl_res = json.load(f)    
# for filename in os.listdir(newpath):
#     if filename.endswith('.json'):
#         with open(os.path.join(newpath, filename), 'r') as f:
#             new = json.load(f)
#         with open(os.path.join(oldpath, filename), 'r') as f:
#             old = json.load(f)

#         for i in range(0, len(new)):
#             idx = 3*i
#             if filename == 'overall_consistency.json':
#                 new[i]['gpt4o_eval'] = old[idx]['multiagent_score']
#                 new[i]['gpt4o_eval']['sora'] = int(sora_eavl_res[str(i)].split(' ')[1][0])
#                 new[i]['human_anno']['sora'] = [int(sora_eavl_res[str(i)].split(' ')[1][0])]

#             tmp = new[i]['human_anno']['sora']
#             new[i]['human_anno'] = old[idx]['human_anno']
#             new[i]['human_anno']['sora'] = tmp
#             new[i]['baseline_score'] = new[i]['gpt4o_eval']
#             new[i]['multiagent_score'] = new[i]['gpt4o_eval']
        
#         with open(os.path.join(newpath, filename), 'w') as f:
#             json.dump(new, f, indent=4)


In [2]:
import numpy as np
import random

def predict_scores(history_matrix, known_score):
    """
    预测评分行。

    参数：
        history_matrix (numpy.ndarray): 已知的历史评分矩阵，形状为 (n, 4)，每行表示一个模型，每列表示一个人的评分。
        known_score (float): 第 4 行的第一个评分 (a4)。

    返回：
        list: 预测的第 4 行评分，包含 4 个元素。
    """
    # 计算每行的评分偏差矩阵
    bias_matrices = []
    for row in history_matrix:
        bias_matrix = np.subtract.outer(row, row)  # 每行的偏差矩阵
        bias_matrices.append(bias_matrix)

    # 统计每对人的偏差范围 (min 和 max)
    bias_min = np.min(bias_matrices, axis=0)  # 每对人的最小偏差
    bias_max = np.max(bias_matrices, axis=0)  # 每对人的最大偏差

    # 预测第 4 行的评分
    predicted_row = [known_score]  # 第一个评分已知
    for i in range(1, 4):  # 预测剩余 3 个评分
        # 随机生成一个偏差，在 [bias_min[0, i], bias_max[0, i]] 范围内
        bias = random.uniform(bias_min[0, i], bias_max[0, i])
        predicted_score = known_score + bias  # 用第一个评分加偏差预测
        # 取整并限制范围
        predicted_score = int(np.clip(round(predicted_score), 1, 5))
        predicted_row.append(predicted_score)

    return predicted_row

In [3]:
dimensions = []
for filename in os.listdir(newpath):
    if filename.endswith('.json'):
        dimensions.append(filename.split('.')[0])

for dim in dimensions:
    with open(os.path.join(newpath, dim+'.json'), 'r') as f:
        new = json.load(f)
    for i in range(0, len(new)):
        human_anno = np.array(list(new[i]['human_anno'].values())[:-1])
        known_score = new[i]['human_anno']['sora'][0]
        new[i]['human_anno']['sora'] = predict_scores(human_anno, known_score)

    with open(os.path.join(newpath, dim + '.json'), 'w') as f:
        json.dump(new, f, indent=4)